<a href="https://colab.research.google.com/github/ahcamachod/1904-word2vec-entrenamiento-de-word-embedding/blob/main/word2vec_entrenamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Word2Vec: Entrenamiento de Word Embedding

En este notebook encontrarás el desarrollo del proyecto para modelar tus propias representaciones de Word Embedding utilizando **Word2Vec** .


Los principales recursos de Python que utilizaremos como base para nuestro modelaje se encuentran en:


*https://spacy.io/*

*https://radimrehurek.com/gensim/models/word2vec.html*


La documentación referente al diseño de las arquitecturas Word2Vec la encontramos en:


*https://arxiv.org/pdf/1301.3781.pdf*

#**<font color=red>EXPLORANDO SPACY</font>**

---



Vamos a:

1°)Obtener Noticias

2°)Realizar la Vectorizacion de los titulares

3°)Clasificar a que categorias pertenecen estas noticias

Con estos elementos vamos a colocar un Modelo en produccion en una API.

##**INICIANDO SPACY**

In [ ]:
# Linea de codigo contruida en el modelo para construir nuestro vocabulario y para ver como lo construyo.

import logging

logging.basicConfig(format="%(asctime)s - %(message)s", level=logging.INFO)

In [ ]:
# 1°) Importamos bibliotecas y nuestros datasets, en mi caso lo importo desde la pc y NO del drive --->

import pandas as pd

noticias_train = pd.read_csv("/content/noticias_entrenamiento.csv")
noticias_test = pd.read_csv("/content/noticias_prueba.csv")

In [ ]:
# 2°) Vemos la forma de nuestro dataset de Entrenamiento y vemos una muestra con 2 de los registros del dataset --->

noticias_train.shape

# Nos devuelve la cantidad de filas y columnas.

(91844, 7)

In [ ]:
noticias_train.sample(2)

# Nos devuelve una muestra del dataset con la variable CATEGORIA que es nuestro target, nuestra variable DEPENDIENTE y el TITULAR que es
# nuestra variable INDEPENDIENTE el cual vamos a estar procesando para llegar a nuestro Target.

,fecha,titulo,pais,extracto,resumen,enlace,categoria
49544,2022-03-30 15:03:01,"La inflación se desboca en España, impulsada p...",FR,"La inflación bate récords en España: 9,8%, su ...","España\n\nEn España, el aumento de los precios...",https://www.rfi.fr/es/europa/20220330-la-infla...,economia
86621,2022-03-22 05:30:00,Cae una mujer de 80 años tras arrancarle la vi...,MX,"Autoridades de Japón, arrestaron a una mujer, ...","Chiba, Japón.- Autoridades de la prefectura de...",https://www.tribuna.com.mx/mundo/2022/3/21/cae...,mundo


In [ ]:
# 3°) Hacemos un SHAPE y un SAMPLE con noticias Prueba(test) --->

noticias_test.shape

(22961, 7)

In [ ]:
noticias_test.sample(2)

# Nos devuelve unas noticias diferentes y todas en español.

,fecha,titulo,pais,extracto,resumen,enlace,categoria
3349,2022-03-29 20:13:37,Montoya tras debate de vacancia: El pueblo es ...,PE,���Este debate sirvi坦 para desenmascarar a los...,"Jorge Montoya, legislador de Renovaci坦n Popula...",https://exitosanoticias.pe/v1/montoya-tras-deb...,politica
3170,2022-03-21 21:33:49,Celebra San Miguel de Allende el equinoccio de...,MX,La llegada del equinoccio de primavera se cele...,"San Miguel de Allende, Guanajuato / 21.03.2022...",https://www.milenio.com/politica/comunidad/cel...,politica


In [ ]:
# 4°) Ahora vamos a la pagina de SPACY para saber como INSTALARLA --> USAGE --> LINUX --> X86 --> PIP --> CPU --> SPANISH -->
# pip install -U pip setuptools wheel, pip install -U spacy(estos 2 ya vienen instalados en Colab) --> solo instalamos:
# python -m spacy download es_core_news_sm, y luego de instalados reiniciamos el ambiente--->

!python -m spacy download es_core_news_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 47.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# 5°) Ahora vemos como se hace la importacion del paquete SPACY que fue instalado --> MODELS --> SPANISH --> import spacy --->
# --> nlp = spacy.load("es_core_news_sm") ---->

import spacy

nlp = spacy.load("es_core_news_sm")

***Defincion:***

***Para saber más: Spacy***

Spacy es una biblioteca de código abierto gratuita para el procesamiento avanzado de lenguaje natural (PLN o también NLP en inglés).

Esta biblioteca fue proyectada específicamente para el uso en producción y ayuda a crear aplicaciones que procesan y “comprenden” grandes volúmenes de textos. Algunos ejemplos de su utilización son para la extracción de información, preprocesamiento y aprendizaje profundo (Deep Learning).

Vamos a utilizarla solamente con los recursos que necesitaremos, pero a continuación dejo el link de la documentación de la biblioteca Spacy para quien desee aprender y descubrir más detalles sobre sus aplicaciones.

##**SPACY Y SUS ESTRUCTURAS**

Vamos a continuar entonces aquí investigando un poco más sobre el paquete spacy y sus estructuras. Vamos a la documentación y vengamos a la opción que dice usage o uso. Entonces nosotros ya instalamos spacy. Veamos aquí un poco más donde dice spaCy 101.

Vamos a darle clic y esta opción nos dice todo lo que necesitamos saber para poder comenzar a trabajar con spaCy. Si ya has trabajado con procesamiento de lenguaje natural entonces debes de estar familiarizado pero si no, a través de esta documentación vas a poder comprender un poco mejor cómo trabajar con este ecosistema.

Entonces nos dice lo que muestra algunos features, dice que hace la tokenización, hace la etiqueta, las partes de la frase entonces le dice por ejemplo si es un verbo, un nombre o adverbio, adjetivo, etcétera. Hacer parsing de dependencia, asignado dependencia sintáctica, etiquetas de dependencia sintáctica describiendo la relación entre tokens individuales como sujeto u objeto, lemmatización, sentence boundary, por ejemplo Named Entity Recognition.

Entonces por ejemplo él le da nombres a objetos del mundo real como personas, compañía o lugares. Y si venimos a donde dice linguistic annotations nos da algunos insights dentro de la estructura gramatical de un texto. Entonces si bajamos un poco más y dice luego de que has descargado e instalado una pipeline entrenada puedes cargarlo utilizando spacy.load, que lo que nosotros hicimos en el video anterior.

Nosotros importamos spacy y ejecutamos el comando spacy.load, al paquete que descargamos que era una pipeline y lo almacenamos en un objeto llamado nlp. Dice que esto nos devuelve un objeto de tipo lenguaje dice aquí con teniendo todos los componentes y los datos necesarios para procesar el texto.

Lo llamamos usualmente nlp. Entonces es este nlp que hay aquí. Cuando llamamos el objeto nlp en un string de texto entonces nos devuelve un doc, un objeto tipo doc procesado. Entonces vean ustedes, aquí dice lo siguiente. Tenemos un ejemplo que nos muestra que en un objeto doc se almacena el objeto nlp al cual se le introduce como parámetro un texto.
Y yo puedo iterar sobre este doc porque este doc lo que tiene es una secuencia de tokens. Y cada token tiene un texto, la parte de la frase, par of speech, qué tipo de palabra es, si es verbo, es pronombre, adverbio y la dependencia.

***PIPELINES:***

Entonces si venimos a la documentación que puedo iterar, para cada token en doc entonces él me puede devolver el texto, en la parte de speech que es, la parte de la frase, o sea qué tipo de palabra es y la dependencia de la misma.

Entonces yo puedo ejecutar este comando y él me va a devolver a mí pues todo este ejemplo. Sigamos averiguando un poco más en la documentación donde dice pipelines y aquí nos explica un poco más qué es lo que sucede cuando yo utilizo el objeto nlp sobre un texto.

Dice: cuando llamamos a nlp sobre un texto spacy lo primero que hace es tokenizarlo para producir un objeto doc. Este doc es procesado en diversos pasos y esto es lo que se conoce como pipeline de procesamiento. Entonces estos pipelines preentrenados, que fue el que nosotros descargamos, ese core news que descargamos al principio, típicamente incluye un tagger, un lemmatizer, un parser y un reconocedor de entidades.

Entonces aquí está lo que es cada uno de ellos. Entonces tokenizers segmenta el texto en tokens. El parser asigna labels de dependencia, etiquetas de dependencia, el ner detecta y etiqueta entidades nombradas que son nombres de personas, gente reconocida y así sucesivamente. Entonces aquí está la gráfica del pipeline que ejecuta el objeto nlp sobre un texto para transformarlo en doc.

In [ ]:
# 1°) Vamos a ver un ejemplo, creamos un texto --->

texto = "Big Data es una ciencia que nos permite trabajar velozmente con grandes volumenes de datos."

In [ ]:
# 2°) Ahora vamos a aplicar NLP al texto y lo vamos a almacenar en una Variable llamada DOC, un objeto DOC que se va a
# crear con NLP del texto --->

doc = nlp(texto)

In [ ]:
# 3°) Si le preguntamos que tipo de objeto es --->

type(doc)

# Es un objeto tipo SPACY TOKENS DOC, y de esta manera nosotros podemos Iterar, nos puede devolver el texto, la frase, que tipo
# de palabra es y la dependencia.

spacy.tokens.doc.Doc

In [ ]:
# 4°) Vamos a ver que tiene DOC, si yo puedo Iterar sobre DOC, yo puedo ver  --->

doc[0]

# Nos devuelve la posicion 0 que es la palabra Big

Big

In [ ]:
# Podemos ver ENTIDAD, identificar entidadas nombradas, sabe que la frase tiene que ver con Big Data:

doc.ents # Metodo

(Big Data,)

In [ ]:
# Podemos preguntar si la posicion 5 es una STOP WORDS, que se detiene las frase en QUE, entonces nos devuelve TRUE:

doc[5].is_stop

True

In [ ]:
# Nos dice FALSE porque NO es una STOPO WORDS porque es un verbo:

doc[9].is_stop

False

In [ ]:
# Nos devuelve que tipo de palabra es:

doc[9].pos_

'ADJ'

Mi objeto DOC esta compuesto por varios TOKENS y estos tokens pueden ser la puntuacion, palabra, numero, etc.

***Defenicion:***

***Spacy: estructura de datos***

Entre las principales ventajas de la biblioteca Spacy está su arquitectura, centralizada en dos estructuras de datos principales: Doc y Vocab.

***Sobre la estructura Doc ¿Cuál de las siguientes opciones es correcta?***

* Doc es un objeto que posee una secuencia de tokens y todas sus anotaciones.

 Doc es un objeto con una secuencia de tokens y anotaciones de acuerdo con los modelos de lenguaje que estés utilizando (“es_core_news_sm”, “en_small”...).

#**<font color=red>PREPROCESAMIENTO CON SPACY</font>**

---



##**PREPROCESAMIENTO DE DATOS**

Vamos a tratar nuestros textos, vamos a crear un FUNCION que nos permita a nosotros identificar a los TOKENS, tomar un STRING separarlos en diversos tokens, quitar las puntuaciones, quitar numeros y ponerlo todo en una LISTA de modo que que podemos trabajar con esta lista e ITERAR, y lo vamos a hacer de forma manual. Tambien debemos colocar todo en MINUSCULA, el texto tiene casi 100 mil titulares y para esto es mejor utilizar un GENERATOR EXPRESSION que es un generador que facilita convertir todas las Mayusculas en Minusculas.

In [ ]:
# 1°) Creamos una Variable que se llame TEXTO para el tratamiento y ahi almacenamos nuestro Generador --->

texto_para_tratamiento = (titulo.lower() for titulo in noticias_train.titulo) # Generador que coloca todos los titulos del noticias_train en minuscula.

In [ ]:
# 2°) Definimos una FUNCION para tratar el texto --->

def trata_texto(doc): # Recibe un doc.
  token_valido = [] # Creamos Lista Vacia para guardar Token que sean validos.
  for token in doc: # Iteramos
  # Para que sea valido, NO tiene que tener STOP WORDS como "que", "y", etc y tiene que ser una palabra(is_alpha):
    valido = not token.is_stop and token.is_alpha
    if valido: # Si es valido
      token_valido.append(token.text) # Si es valido, lo guardamos en la lista vacia.
  if len(token_valido) > 2: # La frase tiene que ser de mas de 2 palabras, para que pueda ver el contexto de esa frase.
      return " ".join(token_valido) # Nos retorna un join de token valido.

In [ ]:
# 3°) Ejecutamos la funcion trata texto al Doc y utilizamos el ejemplo que hicimos anteriormente --->

texto = "Big Data es una ciencia que nos permite trabajar velozmente con grandes volumenes de datos."

doc = nlp(texto)
trata_texto(doc) # Le aplicamos la funcion.

# Nos devuelve un STRING con las palabras pero sin incluir las STOP WORDS como "que", "y" ni tampoco las puntuaciones.

'Big Data ciencia permite trabajar velozmente volumenes datos'

In [ ]:
# 4°) Comprobamos si la FUNCION realmente trabaja, y para esto agregamos numeros, exclamaciones y signos al texto --->

texto = "Big Data 463762983 es una ciencia! que nos permite $%&%$$$ trabajar velozmente con grandes volumenes de datos."

doc = nlp(texto)
trata_texto(doc) # Le aplicamos la funcion.

# Podemos ver que la FUNCION trabaja bien.

'Big Data ciencia permite trabajar velozmente volumenes datos'

##**OPTIMIZANDO EL TRATAMIENTO DE LOS DATOS**

Ya creamos una FUNCION que nos permite tratar nuestro texto para que sean solo palabras, ya tenemos nuestro GENERADOR y ahora vamos lo aplicamos para poner todas las palabras del texto en Minuscula en el objeto DOC.

In [ ]:
# 1°) Para aplicarle la funcion y el generador a todos los titulares vamos a utilizar un LIST COMPREHENSION[], tambien vamos a contabilizar
# cuanto se demora este proceso --->

from time import time # Importamos time para ver demora

t0 = time() # T0= Instancia de time que es en este momento, luego creamo la variable con la list comprehension.

texto_tratado = [trata_texto(doc) for doc in nlp.pipe(texto_para_tratamiento, batch_size=1000)] # aplicamos la funcion y generador.

tf = time() - t0 # Tiempo actual luego de ejecutar la variable de arriba.
print(tf/60) # Para que nos imprima el tiempo que demora en minutos, por eso lo dividimos por 60 seg.

# Nos devuelve el tiempo que tarda la ejecuacion de la variable.

2.710200568040212


In [ ]:
# 2°) Ahora vamos a almacenar nuestros titulos tratados de los titulares en un DataFrame --->

titulos_tratados = pd.DataFrame({"titulo": texto_tratado}) # la variable contiene el el titulo del dataset con la funcion que contiene un funcion y generador.
titulos_tratados.head()

# Nos devuelve los titulos en Minuscula sin las STOP WORDS, sin numeros, sin signos y puntuaciones para todo el data set de Entrenamiento.

,titulo
0,tenso debate senado argentino refinanciamiento...
1,triunfo dramático cruz azul duelo campeones atlas
2,moderar inflación urgente marcha plan calviño
3,llega the batman hbo max mira fecha estreno ex...
4,guzmán default fmi implicaba ajuste caída prod...


In [ ]:
len(titulos_tratados) # nos devuelve la cantidad de filas de texto tratados del dataset de Entrenamiento.

91844


***Generator Expression***


En el siguiente fragmento de código, empleamos el llamado generator expressions:

**texto_para_tratamiento = (titulo.lower() for titulo in noticias_train.titulo)**

En este caso, podemos comparar los generator expressions a las list comprehensions, con la diferencia de que los primeros “crean” sus elementos solamente cuando son invocados. Como el generator expressions produce un item a la vez, este puede economizar bastante el uso de memoria.

* A tratar los datos para entrenar un modelo Word2Vec;
* Cómo usar el objeto doc de Spacy para auxiliar en el preprocesamiento de los datos;
* Cómo paralelizar el preprocesamiento con el recurso nlp.pipe() de Spacy.



#**<font color=red>HIPERPARAMETROS DE WORD2VEC</font>**

---



##**HIPERPARAMETROS WORD2VEC**

Vamos a crear el modelo Word2vec, ya preparamos una funcion para tratar nuestros textos y ya tenemos nuestros titulares listos, ahora comenzamos con nuestro modelaje.

In [ ]:
# 1°) Importamos Word2Vec y luego creamos el Modelo --->

from gensim.models import Word2Vec

#modelo_w2v = Word2Vec(sg=0, vector_size=300, window=2)

# sg=0 viene CBOW por defecto(0), (1)=SKIP-GRAM, size=Vector con 300 posiciones, WINDOW= Ventana, 2 palabras antes y despues de la palabra que
# quiero predecir.

##**AVANZANDO EN LOS HIPERPARAMETROS**

Vamos a ver el siguiente problema, supongamos que yo tengo las palabras : ***Alura - Aluras - Aura - Alure***, si yo al momento de modelar necesito solamente la palabra ***Alura*** y NO necesitamos que se repita el resto de las palabras que son incorrectas, para esto necesitamos establecer un PARAMETRO que me permita a mi establecer un limite minimo de palabras para que tenga en cuenta al momento del procesamiento, este parametro es el que se conoce como ***MIN_COUNTS***

In [ ]:
# 1°) Colocamos ese parametro en el modelo --->

modelo_w2v = Word2Vec(sg=0, vector_size=300, window=2, min_count=5, alpha = 0.03, min_alpha=0.0007)

# MIN_COUNTS=conteo minimo de una palabra, ALPHA=Para llegar el minimo global para que el Algoritmo generalice bien, MIN_ALPHA=Valor minimo de Alpha


**Definicion:**

***Hiperparámetros***

Los hiperparámetros son parámetros que configuran la forma que tu modelo será entrenado, por ello se deben establecer antes de la fase de entrenamiento.

***Sobre los hiperparámetros de la implementación gensim del modelo Word2Vec:***

* MIN_ALPHA es la tasa de aprendizaje mínima, cuando se aplica al decaimiento de alpha, configuramos min_alpha para garantizar que alpha no sea menor que min_alpha..

*
El hiperparámetro SG configura la arquitectura de entrenamiento (Skip-Gram o CBOW) del modelo Word2Vec, sg=1 usa la arquitectura Skip-Gram y sg=0 emplea CBOW.

##**VOCABULARIO Y WORD2VEC**

Vamos a construir nuestro VOCABULARIO.

In [ ]:
# 1°) Tomamos el modelo y le aplicamos el metodo BUILD_VOCAB --->

# modelo_w2v.build_vocab() # El primer parametro que tenemos que adicionar al metodo BUILD_VOCAB es SENTENCE= a una lista de tokens

In [ ]:
# 2°) Creamos la LISTA DE TOKENS para el parametro con LIST COMPREHENSIONS para BUILD_VOCAB --->

titulos_tratados = titulos_tratados.dropna().drop_duplicates() # (a)

lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]

# Nos da un ERROR en SPLIT, nos dice que tenemos que NO se puede aplicar este atributo porque tenemos registro IS NULL!

In [ ]:
# 3°) Vemos cuanto NULOS tenemos --->

#titulos_tratados.isnull().value_counts()

# Nos muestra 1119 Nulos, tenemos que borar estos nulos y tambien los duplicados.

In [ ]:
# 4°) Creamos una variable titulos_tratados y almacenamos la variable titulos_tratados haciendo el DROP en nulos y duplicados --->

# (a) titulos_tratados = titulos_tratados.dropna().drop_duplicates() # Colocamos la variable antes de lista TOKENS

len(titulos_tratados)
# Nos quedaron 82707 titulos sin null y duplicate de los 91844 que teniamos

82487

In [ ]:
# 5°) Ahora construimos nuestro VOCABULARIO con las LISTA TOKENS incluida como parametro de SENTENCES --->

modelo_w2v.build_vocab(lista_lista_tokens)

# NOS CONSTRUYE EL VOCABULARIO.

In [ ]:
# 6°) Ahora vamos a VISUALIZAR que fue lo que hizo el metodo BUILD_VOCAB del modelo modelo_w2v(cbow), pero esta linea de codigo
# va al COMIENZO del notebook , en la primer linea --->

#import logging

#logging.basicConfig(format="%(asctime)s - %(message)s", level=logging.INFO)

# Nos devuelve un LOG en la linea del modelo que construye nuestro vocabulario, con el formato que le asignamos como el dia,
# la fecha y el mensaje a la derecha... colecto todas las palabras y mi cuenta...etc.



***Para saber más: Word2Vec***

Los modelos de representación de palabras, que de alguna forma capturan el contexto en el cual las palabras son utilizadas, fueron un gran avance para el área de procesamiento de lenguaje natural.

Si deseas conocer la parte matemática que se desarrolla tras el modelo Word2Vec, te recomiendo la lectura del paper original de Tomas Mikolov.

Además, puedes acceder al Embbeding Project de Tensorflow para explorar de forma visual una representación Word2Vec y comprender aún más sobre esta técnica.

#**<font color=red>ENTRENAMIENTO DE WORD2VEC</font>**

---
Vamos a Entrenar el modelo WORD2VEC para la arquitectura de CBOW : toma el contexto del titular para predecir la palabra.

##**ENTRENAMIENTO CBOW**

In [ ]:
# 1°) Primero vamos a dar un DIR al modelo para saber cuantos Features o Atributos tiene --->

dir(modelo_w2v)

# Nos devuelve un monton de Atributos pero el que nos interesa es el TRAIN porque debemos entrenar nuestro modelo, tambien
# tenemos otro parametro importante que es CORPUS_COUNT.

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_corpus_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_next_alpha',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_raw_word_count',
 '_save_specials',
 '_scan_vocab',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'add_lifecycle_event',
 'add_null_word',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'cbow_mean',
 'comment',
 'compute_loss',
 'corpus_count',
 

In [ ]:
# 2°) Vamos a Entrenar el modelo y recibe como parametro LISTA_LISTA_TOKENS, el Corpus_File, TOTAL_EXAMPLES= la cuenta de la cantidad
# de las frases que va a ser igual al CORPUS_COUNT y las EPOCHS= que son el numero de Iteraciones sobre el Corpus --->

modelo_w2v.train(lista_lista_tokens, total_examples=modelo_w2v.corpus_count, epochs=30)

# Entreno 16598820 palabras crudas y palabras efectivas.

(14593202, 16598820)

In [ ]:
# 3°) Vamos a ver como generalizo nuestro entrenamiento del modelo --->

modelo_w2v.wv.most_similar("google") # wv=word vector, le preguntamos palabra semejante a google

# Nos devuelve la palabra y el % de confianza de la palabra similar.
# Estamos trabajando con CBOW el toma el contexto para descubrir la palabra.


[('maps', 0.6684199571609497),
 ('chrome', 0.5521437525749207),
 ('comandos', 0.5513606667518616),
 ('android', 0.5427801609039307),
 ('emojis', 0.529778242111206),
 ('store', 0.5147353410720825),
 ('youtube', 0.5113668441772461),
 ('zoom', 0.5095242261886597),
 ('apple', 0.5088775753974915),
 ('apps', 0.5019453763961792)]

In [ ]:
# 4°) Vamos a ver como generalizo nuestro entrenamiento del modelo --->

modelo_w2v.wv.most_similar("microsoft")

[('intel', 0.704036295413971),
 ('nvidia', 0.6934400796890259),
 ('nothing', 0.691305935382843),
 ('ubisoft', 0.6676716208457947),
 ('macbook', 0.6125083565711975),
 ('inc', 0.6100292205810547),
 ('valve', 0.601817786693573),
 ('legends', 0.6007205843925476),
 ('portátiles', 0.598865807056427),
 ('dyson', 0.5903044939041138)]

In [ ]:
# 5°) Vamos a ver como generalizo nuestro entrenamiento del modelo con respecto al deporte --->

modelo_w2v.wv.most_similar("barcelona")

[('guaireña', 0.5547839999198914),
 ('mazatlán', 0.5319907069206238),
 ('barça', 0.5109214782714844),
 ('fc', 0.4946800172328949),
 ('ousmane', 0.48212990164756775),
 ('jude', 0.47585150599479675),
 ('saltillo', 0.4523231089115143),
 ('laporta', 0.4457821547985077),
 ('tapatío', 0.4321860074996948),
 ('erling', 0.4263615608215332)]

In [ ]:
# 6°) Vamos a ver como generalizo nuestro entrenamiento del modelo con respecto al deporte --->

modelo_w2v.wv.most_similar("messi")

[('lionel', 0.7293429970741272),
 ('scaloni', 0.6510369777679443),
 ('roccuzzo', 0.6449397206306458),
 ('antonela', 0.6356933116912842),
 ('stegen', 0.6066796183586121),
 ('neymar', 0.5930041670799255),
 ('leo', 0.5732771754264832),
 ('silbidos', 0.5675395727157593),
 ('psg', 0.5526174306869507),
 ('lewandowski', 0.5500613451004028)]

In [ ]:
# 6°) Vamos a ver como generalizo nuestro entrenamiento del modelo con respecto a auto --->

modelo_w2v.wv.most_similar("ferrari")

[('leclerc', 0.7700124979019165),
 ('bahrein', 0.7064856886863708),
 ('verstappen', 0.6950523257255554),
 ('sainz', 0.6646941304206848),
 ('baréin', 0.6343582272529602),
 ('position', 0.6059914231300354),
 ('pole', 0.5921818614006042),
 ('yeda', 0.570076048374176),
 ('barguil', 0.5683915615081787),
 ('arabia', 0.5592194199562073)]

* Nuestro modelo aprendio a travez de la arquitectura de CBOW con un corpus no muy grande de 9 mil palabras y generalizo bastante bien entrenando solo con los titulares de las noticias.

 De esta manera Entrenamos el modelo con la arquitectura de CBOW que toma el contexto para predecir la palabra.

##**ENTRENAMIENTO SKIP-GRAM**

Ahora vamos a Entrenar el modelo con la Arquitectura de SKIP-GRAM.

In [ ]:
# 1°) Copiamos codigos e instanciamos el modelo pero con SKIP-GRAM --->

# SG=1, WINDOWS=5 ventanas mas grande porque estamos tomando la palabra para predecir el contexto:
modelo_w2v_sg = Word2Vec(sg=1, vector_size=300, window=5, min_count=5, alpha = 0.03, min_alpha=0.0007) # Modelo con Parametros

modelo_w2v_sg.build_vocab(lista_lista_tokens, progress_per=5000) # Construccion del vocabulario del modelo

modelo_w2v_sg.train(lista_lista_tokens, total_examples=modelo_w2v_sg.corpus_count, epochs=30) # Entrenamiento del modelo

(14593075, 16598820)

In [ ]:
# 2°) Ahora vamos a ver como generalizo SKIP-GRAM, copiamos las mismas consultas que hicimos con CBOW y comparamos las
# respuestas de las 2 arquietecturas del modelo --->

# CBOW:
modelo_w2v.wv.most_similar("ferrari")

# Nos devuelve LECLARC con 0.75 % ...

[('leclerc', 0.7700124979019165),
 ('bahrein', 0.7064856886863708),
 ('verstappen', 0.6950523257255554),
 ('sainz', 0.6646941304206848),
 ('baréin', 0.6343582272529602),
 ('position', 0.6059914231300354),
 ('pole', 0.5921818614006042),
 ('yeda', 0.570076048374176),
 ('barguil', 0.5683915615081787),
 ('arabia', 0.5592194199562073)]

In [ ]:
# SKIP_GRAM:
modelo_w2v_sg.wv.most_similar("ferrari")

# Nos devuelve SAINZ con 0.69 % ...

[('sainz', 0.7000979781150818),
 ('leclerc', 0.6752962470054626),
 ('bahrein', 0.6399394273757935),
 ('position', 0.6146969795227051),
 ('baréin', 0.6063476204872131),
 ('bull', 0.5910671949386597),
 ('verstappen', 0.5853387117385864),
 ('yeda', 0.5136747360229492),
 ('charles', 0.5112325549125671),
 ('ocon', 0.5073619484901428)]

In [ ]:
# CBOW:
modelo_w2v.wv.most_similar("google")

# Nos devuelve MAPS con 0.65 % ...

[('maps', 0.6684199571609497),
 ('chrome', 0.5521437525749207),
 ('comandos', 0.5513606667518616),
 ('android', 0.5427801609039307),
 ('emojis', 0.529778242111206),
 ('store', 0.5147353410720825),
 ('youtube', 0.5113668441772461),
 ('zoom', 0.5095242261886597),
 ('apple', 0.5088775753974915),
 ('apps', 0.5019453763961792)]

In [ ]:
# SKIP-GRAM:
modelo_w2v_sg.wv.most_similar("google")

# Nos devuelve MAPS con 0.70 %...
# Nos devuelve un contexto un poco mejor de palabras.

[('maps', 0.7099429368972778),
 ('doodle', 0.5738242268562317),
 ('chrome', 0.5727046132087708),
 ('caffarena', 0.5116414427757263),
 ('gmail', 0.5052133202552795),
 ('chromebook', 0.4589812755584717),
 ('búsquedas', 0.45614492893218994),
 ('desenfocar', 0.4531976282596588),
 ('roku', 0.4468194544315338),
 ('apk', 0.44516003131866455)]

* **CBOW :** Tomo el contexto para descubrir la palabra.
* **SKIP-GRAM :** Tomo la palabra para descubrir el contexto.

**Cual Usar?? :** Entonces hay varias ventajas y desventajas. Ellas se encuentran en el documento de Tomas Mikolov.

Aquí él dice por ejemplo que la ventaja utilizando CBOW es que es más rápido y Skip Gram pues se demora más pero Skip Gram puede que generalice mejor tomando un contexto más grande. Sin embargo cada planteamiento dependiendo del proyecto puede ser mejor o más útil o más efectivo que el otro.

In [ ]:
# 3°) Ahora vamos a tomar los 2 modelos y los vamos a almacenar en un archivo de texto para posteriormente poder
# cargar nuestro modelo --->

# CBOW:
modelo_w2v.wv.save_word2vec_format("modelo_cbow_300.txt", binary=False)

# SKIP-GRAM:
modelo_w2v_sg.wv.save_word2vec_format("modelo_sg_300.txt", binary=False)

# Nos aparace cuando lo guardamos en colab.

***Word2Vec: flujo de entrenamiento***

Word2Vec.

Los hiperparámetros son parámetros que configuran la forma como el modelo será entrenado, por ello deben ser informados antes de la fase de entrenamiento.

***Sobre los hiperparámetros de la implementación gensim del modelo Word2Vec:***

* Configuración del modelo, construcción del vocabulario y entrenamiento.


Primero, configuramos el modelo. Después, construimos el vocabulario a partir del corpus y, finalmente, entrenamos la representación Word2Vec.

#**<font color=red>VECTORIZANDO DATOS DE ENTRENAMIENTO Y PRUEBA</font>**

---

Estamos iniciando con el Clasificador de Noticias que toma los Titulares y nos entrega como OUTPUT la categoria a la cual corresponde la noticia.



##**INICIANDO EL CLASIFICADOR**

Nosotros desarrollamos una Funcion TRATA TEXTO que lo que hace es tomar un String, la convertimos en un objeto DOC y a este objeto DOC le aplicamos diversos operaciones para poder quitarle la Puntuacion, lo que NO es palabras y quitar cualquier tipo de Simbolo como asi tambien las Stop Words.

In [ ]:
# 1°) Copiamos la Funcion TRATA_TEXTO, como NO vamos a trabajar con NLTK ni con STRING, vamos a trabajar con la biblioteca SPACY,
# la cargamos y luego pondemos algunas parametros de cosas que NO queremos que utilice --->

import spacy

nlp = spacy.load("es_core_news_sm", disable=["parser", "tagger", "ner", "textcat"])

# Funcion:
def tokenizador(texto): # Cambiamos nombre de la Funcion y ahora recibe texto.
  doc = nlp(texto) # Creamos el objeto DOC que recibe nlp texto.
  token_valido = [] # Creamos Lista Vacia para guardar Token que sean validos.
  for token in doc: # Iteramos
  # Para que sea valido, NO tiene que tener STOP WORDS como "que", "y", etc y tiene que ser una palabra(is_alpha):
    valido = not token.is_stop and token.is_alpha
    if valido: # Si es valido
      token_valido.append(token.text.lower()) # Si es valido, lo guardamos en la lista vacia.
  return token_valido


# Ya tenemos nuestro TOKENIZADOR.

In [ ]:
# 2°) Vamos a tomar un STRING de un TITULAR para probar la funion --->

texto = noticias_train.titulo.loc[41958]
tokens = tokenizador(texto) # pasamos la funcion tokenizador al texto.
print(tokens)

# La funcion trabaja bien y nos devuelve los TOKENS limpios.

['frutas', 'ayudan', 'bajar', 'peso']


##**COMBINACION DE VECTORES**

Vamos a realizar la combinacion de cada una de las palabras para generar un VECTOR que represente a cada uno de los TITULARES de noticias, pero para esto tenemos que importar mis modelos de KEYEDVECTORS que almacenamos.

In [ ]:
# 1°) Importamos KeyedVectors, Numpy(para las matrices) y luego creamos nuestros Modelos --->

from gensim.models import KeyedVectors
import numpy as np

modelo_cbow = KeyedVectors.load_word2vec_format("/content/modelo_cbow_300.txt") # load_word2vec_format= para cargar texto.
modelo_sg = KeyedVectors.load_word2vec_format("/content/modelo_sg_300.txt")

In [ ]:
# 2°) Ahora creamos la Funcion COMBIANCION DE VECTORES POR SUMA, que recibe los Tokens y el Modelo que vamos a usar --->

def combinacion_vectores_por_suma(tokens, modelo):
  vector_resultante = np.zeros((1,300)) # Vector Numpy con una matriz de 1,300.
  for token in tokens: # Para cada uno de los token en Tokens
    try: # Intenta
      vector_resultante += modelo.get_vector(token)
    except KeyError:
      continue
  return vector_resultante

In [ ]:
# 3°) Probamos la Funcion ---->

vector_texto = combinacion_vectores_por_suma(tokens, modelo_cbow) # Aplicamos la Funcion al texto que veniamos utilizando[41958]
print(vector_texto.shape) # Tamaño.
print(vector_texto) # Representacion Vectorial

# Nos devuelve el tamaño y representacion vectorial del titular 41958(vector del titular).

(1, 300)
[[-0.1783093   0.4551409   1.20113603  0.23892139 -0.28846563  0.50915633
   0.8994507  -0.84211266  0.96918501  0.65691405 -1.65884745 -0.6130172
  -1.77031472  2.9145195   0.03093125 -0.31444959 -0.2979071   0.75660521
  -1.12727798 -0.38963054 -0.20067446 -2.05806889 -0.26897394 -0.89388443
   0.48939249 -0.5873139  -1.49229313  0.04147832  1.79037429  0.20498748
   0.33093708  0.38423674  1.29645954 -0.16475333 -0.0718601  -2.04550698
  -0.68330836 -1.0583688   0.26943581 -1.30475834  0.22440809 -0.5409074
  -0.15102907 -0.30915844  0.47543196  0.36488396 -1.84529944 -1.55044336
   0.0689904   1.36901489 -0.24421154 -0.71069192 -0.97837563  0.06932724
  -1.0329171  -1.04462859  1.43282366  0.67525192 -0.01121043  0.08296661
   1.87076334  1.49012609 -0.54295328  1.70105469 -0.57420567  0.65926257
  -0.4942462   2.3551878  -1.3041252   0.83211318  0.07754895  2.15501651
   3.49465731  1.55264395  1.34678543 -0.74081644  0.10175785  1.98147202
  -0.10415301  0.51462108  2.61

**Definicion:**

***El Tokenizador***

Hemos explorado y desarrollado funciones auxiliares para utilizar en la creación del clasificador. Entre las funciones desarrolladas, tenemos:

* La responsable por el tratamiento y la tokenización de los textos

* La que combina los vectores de cada palabra y crea una representación vectorial para toda la frase

*La función que crea la representación vectorial de toda la base de datos.

***Si una de las funciones recibe como entrada:***

"¡Río de Janeiro 1231231 ***** @#$ es una ciudad maravillosa!"

***Y entrega como salida la siguiente lista:***

['río', 'janeiro', 'ciudad', 'maravillosa']

***Nota: considera los modelos nlp entrenados en Español:***

nlp = spacy.load("es_core_news_sm", disable=["parser", "ner", "tagger", "textcat"])

***Se trata de la función:***


    def tokenizador(texto):

    doc = nlp(texto)
   
    tokens_validos = []
   
    for token in doc:
       
      valido = not token.is_stop and token.is_alpha
       
     if valido:
           
         tokens_validos.append(token.text.lower())

    return  tokens_validos

**Aquí estamos removiendo los stop words y caractéres no alfabéticos, y dejando palabras en minúscula, retornando exactamente lo que fue solicitado.**   

##**VECTORIZANDO LOS TITULARES**

Vamos a Vectorizar todos los textos.

In [ ]:
# 1°) Finalmente crearemos las matrices de titulares para prueba y entrenamiento de nuestras 2 arquitecturas Word2Vec y haremos
# un print() a sus formas respectivas. (Nos apoyaremos en la función matriz() también desarrollada en el curso de
# introducción a Word2Vec) --->

t0 = time()# Para calcular lo que tarda la Funcion en ejecutar.

def matriz(textos,modelo): # Funcion recibe texto y modelo.
  x = len(textos) # En X la Longitud de texto que son los Titulares.
  y = 300 # Estamos trabajando con un vector denso de 300 posiciones.
  matriz = np.zeros((x,y)) # Creamos la Matriz
  for i in range(x):
    palabras = tokenizador(textos.iloc[i]) # En palabras vamos a ejecutar la Funcion TOKENIZADOR en la cual van a entrar nuestros textos
    # y va a localizar cada Subindice[i] que se encuentra dentro de X.
    matriz[i] = combinacion_vectores_por_suma(palabras, modelo) # En la posicion de nuestra Matriz en cada Iteracion va a ejecutar la Funcion COMBINACION...
    # que recibe a su vez las palabras y el modelo.
  return matriz

# Ahora cramos la Matriz Entrenamiento y Test para el modelo de CBOW y SKIP-GRAM:
matriz_train_cbow = matriz(noticias_train.titulo, modelo_cbow)
matriz_test_cbow = matriz(noticias_test.titulo, modelo_cbow)

matriz_train_sg = matriz(noticias_train.titulo, modelo_sg)
matriz_test_sg = matriz(noticias_test.titulo, modelo_sg)

tf = time()-t0

print(tf/60)
print(matriz_train_cbow.shape)
print(matriz_test_cbow.shape)
print(matriz_train_sg.shape)
print(matriz_test_sg.shape)

# Demoro 17 minutos par vectorizar los titulares de los 2 modelos.

14.749051038424174
(91844, 300)
(22961, 300)
(91844, 300)
(22961, 300)


**Explicacion**

***¿Entonces qué va a hacer nuestra función matriz?***  Ella va a recibir como parámetros nuestros titulares, nuestros textos, y va a recibir también como parámetro el modelo. Entonces recordemos que vamos a entrenar los dos modelos, tanto el modelo cbow como el modelo Skip Gram.

Entonces ella se crea según sus dimensiones que vas hacer el valor de X va a ser la cantidad de filas o la cantidad de titulares que hay en nuestra matriz y el valor de y van a ser las 300 posiciones de nuestro vector denso. Entonces para cada uno de estos titulares entonces serían una variable llamada palabras va aplicar la función tokenizador que es la función que ella crea nuestro objeto nlp.

Nosotros lo que hicimos aquí fue eliminar algunos de los pasos o deshabilitar alguno de los pasos como el parser, el tagger, el ner y el textcat, entonces ella va a eliminar todos estos pasos para crear el objeto nlp y sobre el objeto nlp se aplican todas las transformaciones que mencionamos previamente.

Después entonces hace la combinación de vectores por suma a todas nuestras palabras utilizando también el modelo, que aquí es donde viene ya utilizar bien sea CBOW o Skip Gram y nos devuelve entonces una matriz. Esto se va a demorar bastante tiempo. Inclusive podemos también usar el parámetro time como lo usamos anteriormente.

Veamos al momento de optimizar el tratamiento de los datos, nosotros utilizamos time. Vamos a copiar esta función time, vamos a traerla a vectorizando los titulares y vamos a pegarla. Como ya time estaba importado, no necesitamos importarlo de nuevo. Simplemente vamos a instanciar nuevamente el tiempo.

Entonces de esta forma nosotros vamos ya a tomar nuestro horario en este momento. Al final simplemente nos va a imprimir cuánto se demoró. Entonces vamos a cerrar aquí y vamos a ejecutar esta celda y vamos a dejarla rodar.

**Definicion:**

***La Matriz de vectores***

Hemos explorado y desarrollado funciones auxiliares para utilizar en la creación del clasificador. Entre las funciones desarrolladas, tenemos:

* La responsable por el tratamiento y la tokenización de los textos.

* La que combina los vectores de cada palabra y crea una representación vectorial para toda la frase.

* La función que crea la representación vectorial de toda la base de datos.

***Sobre la función que crea la representación vectorial de toda la base de datos:***

* Primero se crea la matriz con las dimensiones especificadas para sus filas (x) y columnas (y), después se itera en el rango de las filas, se aplica la función tokenizador() a los textos y se almacena en una variable llamada palabra y esta se convierte en uno de los parámetros que recibe la función combinacion_vectores_por_suma() para finalmente devolver la matriz de vectores.



#**<font color=red>ENTRENAMIENTO E INTEGRACION DEL CLASIFICADOR</font>**

---

##**CLASIFICANDO LOS TITULARES**



In [ ]:
# 1°) Importamos nuestra REGRESION LOGISTICA Y CLASSIFICATION_REPORT --->

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [ ]:
# 2°) Definimos la Funion CLASIFICADOR --->

def clasificador(modelo, X_train, y_train, X_test, y_test):
#En la funcion Instanciamos Regression Logistic:
  RL = LogisticRegression(max_iter=1500, n_jobs=-1)#N_jobs=maxima capacidad de procesamiento que ofrece google colab.
  RL.fit(X_train, y_train) # Entrenamos el modelo
  categorias = RL.predict(X_test) # Creamos variable para que almacene el Predict conforme a la matriz de Prueba.
  resultados = classification_report(y_test, categorias) # Creamos variable, instanciamos el reporte que recibe Prueba y categoria.
  print(resultados)
  return RL

In [ ]:
# 3°) Instanciamos nuestro clasificador --->

# Resultados generados antes de corregir la funcion tokenizador(minusculas):
t0 = time()
RL_cbow = clasificador(modelo_cbow, matriz_train_cbow, noticias_train.categoria, matriz_test_cbow, noticias_test.categoria)
tf = time()-t0
print(tf/60)

# Resultado 54%

                      precision    recall  f1-score   support

ciencia y tecnologia       0.61      0.55      0.58      2864
            deportes       0.70      0.75      0.72      4277
            economia       0.60      0.69      0.64      5461
     entretenimiento       0.62      0.59      0.61      3580
               mundo       0.61      0.61      0.61      2838
            politica       0.61      0.51      0.56      3941

            accuracy                           0.63     22961
           macro avg       0.62      0.62      0.62     22961
        weighted avg       0.63      0.63      0.62     22961

2.2637718518575034


**RTA:**

El resultado es de una EXACTITUD(accuracy) del 0.54(54%), resultado MALO!, esto se debe a que cuando cargamos SPACY desabilitamos algunas funcionalidades para que sea mas rapida la ejecucion, pero hay algo que NO hicimos a nuestro preprocesamiento y que lo habiamos hecho anteriormente que fue colocar todo en MINUSCULA en la funcion TOKENIZADOR, entonces debemos agregarlo:

***token_valido.append(token.text) ---> token_valido.append(token.text.lower())***

Y como la funcion TOKENIZADOR se usa en la funcion MATRIZ debemos volver a ejecutar ambas funciones nuevamente, para luego volver a ejecutar la funncion Clasificador y comparar los resultados luego de agregar minusculas.

In [ ]:
# Volvemos a INSTANCIAR Regresion Logistica --->

t0 = time()
RL_cbow = clasificador(modelo_cbow, matriz_train_cbow, noticias_train.categoria, matriz_test_cbow, noticias_test.categoria)
tf = time()-t0
print(tf/60)

# Podemos ver que nos devuelve una EXACTITUD(accuracy) del 63 % , el hecho de haber colocado todo en MINUSCULA si era parte
# del preprocesamiento necesario para generalizar mejor!

                      precision    recall  f1-score   support

ciencia y tecnologia       0.61      0.55      0.58      2864
            deportes       0.70      0.75      0.72      4277
            economia       0.60      0.69      0.64      5461
     entretenimiento       0.62      0.59      0.61      3580
               mundo       0.61      0.61      0.61      2838
            politica       0.61      0.51      0.56      3941

            accuracy                           0.63     22961
           macro avg       0.62      0.62      0.62     22961
        weighted avg       0.63      0.63      0.62     22961

2.019532104333242


##**COMPARANDO LOS MODELOS**

In [ ]:
# 1°) Cambiamos el codigo de CBOW a SG --->

t0 = time()
RL_sg = clasificador(modelo_sg, matriz_train_sg, noticias_train.categoria, matriz_test_sg, noticias_test.categoria)
tf = time()-t0
print(tf/60)

# Obtuvimos una EXACTITUD del 64 %, si comparams con CBOW que obtuvo el 63 %, pero vemos que se demoro mucho menos, pero recordemos
# que en SKIP-GRAM tomamos la palabra para decifrar el contexto, tambien tomamos una ventana mayor que nos ayudo mas a que el modelo
# pueda convergir mejor.

                      precision    recall  f1-score   support

ciencia y tecnologia       0.64      0.56      0.60      2864
            deportes       0.72      0.76      0.74      4277
            economia       0.62      0.71      0.66      5461
     entretenimiento       0.64      0.61      0.62      3580
               mundo       0.61      0.63      0.62      2838
            politica       0.63      0.53      0.57      3941

            accuracy                           0.64     22961
           macro avg       0.64      0.63      0.64     22961
        weighted avg       0.64      0.64      0.64     22961

0.8049120942751566


***CONCLUSION:***

***¿Entonces qué podemos concluir de lo que hicimos?*** Porque en el curso anterior si nosotros venimos al curso anterior, a los resultados del curso anterior vean ustedes que con el modelo que cargamos de Word2Vec, que era un modelo más holístico, un modelo entrenado con billones de palabras, nos dio un resultado inclusive digamos menos acertado que el resultado que obtuvimos ahorita con el modelo que entrenamos de Skip Gram con muchísimos menos datos.

¿Esto porque será? Entonces nosotros entrenamos nuestro modelo directamente considerando el contexto en el cual estábamos trabajando que era un contexto de noticias, de modo que se facilitó mucho esta clasificación porque trabajamos con unos datos que ya tenían de una cierta forma todo el contenido que facilitaría la clasificación.

Entonces pues realmente el resultado sí es mejor utilizando directamente nuestro propio WordEmbedding, creado a través pues desde todos los códigos que ejecutamos a lo largo de nuestro entrenamiento. Otro aspecto muy importante es que el modelo que utilizamos en el curso anterior, él tomaba como referencia muchas bases de datos, Wikipedia, noticias, artículos, documentales, entrevistas, de todo.
Nosotros aquí fuimos más enfocados en el contenido que nos llamaba la atención o en el contenido de nuestro proyecto que era las noticias directamente. Entonces todo ello facilitó trabajar de una forma más específica porque cuando creamos nuestro propio modelo de Word Embedding, entonces nosotros podemos trabajar de una manera más específica de acuerdo con nuestras necesidades y así nuestros resultados van a ser mucho mejores.

Ahora con nuestros modelos listos ya con lo que hemos venido hablando debemos proceder entonces a hacer un dump de nuestros modelos para poderlo utilizar por ejemplo de una API que sea útil a través de una aplicación web. Entonces para ello en una celda nueva de código vamos a hacer importa pickle, y ahora vamos a crear nuestros archivos pickle.

In [ ]:
# 2°) Ahora con nuestro modelos listos vamos a hacer un DUMP para poder utilizarlos en una API, en una aplicacion web --->

import pickle

with open("lr_cbow.pkl", "wb") as f:
  pickle.dump(RL_cbow, f)

with open("lr_sg.pkl", "wb") as f:
  pickle.dump(RL_sg, f)